In [26]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

from numba import njit, jit
from numpy.linalg import norm

from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import lr_scheduler
from torchmetrics import MeanAbsolutePercentageError

import seaborn as sns
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

from utility_funcs import *
from LJ_modeling_realization.includes.constants import *

import dill

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, ConstantKernel, RBF
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import Normalizer

In [27]:
N = CFG.N

In [28]:
# dt = float(input("integrated dt:"))

In [29]:
model_type = "gpr"

In [30]:
class SklearnModel:
    def __init__(self, model):
        '''
        model - sklearn model
        '''
        self.model = model

    def __call__(self, X):
        return self.model.predict(X)

In [31]:
model_file_path = f'./trained_models/{model_type}_2_movements_K3.pickle'
model_vel_file_path = f'./trained_models/{model_type}_2_velocities_K3.pickle'

# Сетка предсказания сжатые будет выдавать и их надо будет возвращать к обычному скейлу
descaler_path = f'./trained_models/descaler_{CFG.N}_K{CFG.K}.pickle'
descaler_vel_path = f'./trained_models/descaler_vel_{CFG.N}_K{CFG.K}.pickle'

In [32]:
with open(model_file_path, 'rb') as handle:
    model = dill.load(handle)

with open(model_vel_file_path, 'rb') as handle:
    model_vel = dill.load(handle)

if model_type == "gpr":
    descaler = Descaler(1, 0)
    descaler_vel = Descaler(1, 0)
    model = SklearnModel(model)
    model_vel = SklearnModel(model_vel)
else:
    with open(descaler_path, 'rb') as handle:
        descaler = dill.load(handle)
    

In [33]:
def csv_row_to_state(coords_path, vels_path, row_number=0):
    '''
    This function will be mostly used to start integration and create header for csv to write into
    '''

    row_coords = np.array(pd.read_csv(
        coords_path
    ).iloc[row_number, :][1:]).reshape(CFG.N, 3)

    row_vels = np.array(pd.read_csv(
        vels_path
    ).iloc[0, :][1:]).reshape(CFG.N, 3)
    
    state = {i: [row_coords[i], row_vels[i]] for i in range(CFG.N)}


    return state

In [34]:
n_state = csv_row_to_state(
    f'./LJ_modeling_realization/folded_coords{CFG.N}.csv',
    f'./LJ_modeling_realization/velocities{CFG.N}.csv',

    row_number=-1   # НАЧИНАЕМ ПРЕДСКАЗЫВАТЬ С КОНЦА ИЗВЕСТНОЙ ТРАЕКТОРИИ
)

In [35]:
def get_distances(n_state, atom_numb):
    '''
    Возвращает все расстояния от данного атома
    '''
    pass

In [36]:
# @njit
def get_closest_atom(state, atom_number):
    other_atom_numbers = [i for i in range(CFG.N) if i != atom_number]
    # print(other_atom_numbers)
    Rel_matrix = np.zeros([CFG.N - 1, 3])

    for i, other_numb in enumerate(other_atom_numbers):
        Rel_matrix[i] = state[atom_number][0] - state[other_numb][0]

    # Getting the closest atom to atom_number:
    dists = norm(Rel_matrix, axis=1)

    closest_num = np.argmin(dists)

    closest_num += int(closest_num >= atom_number)   # Тут же пропущен индекс текущего атома - и надо сдвинуть типо

    return closest_num

In [37]:
# get_closest_atom(n_state, 10)

ошибка в функции сверху ща: оно индекс не тот возвращает

In [38]:
def _get_relative_positions(state, atom_number):
    '''
    This function processes one row of csv into something that we can work with

    Returns np.array matrix that consists of relative positions vectors for passed atom_number to every other atom
    and then we can chose only closest N_neighbours in the next functions
    
    row: df.iloc[row] - typeof(row): pd.Series
    
    returns: Rel_matrix, f_vec
    '''

    other_atom_numbers = [i for i in range(2) if i != atom_number]
    Rel_matrix = np.zeros([len(other_atom_numbers), 3])

    for i, other_numb in enumerate(other_atom_numbers):
        Rel_matrix[i] = state[atom_number][0] - state[other_numb][0]

    return np.array(Rel_matrix)

def create_V_i(i, normalized_m, norms, r_cut=CFG.r_cut, p=CFG.p):
    '''
    normalized_m: matrix of relative distances, where rows - normalized vectors
    i: i-th component of r_cut and p, i in range 1..K (or in 0..K-1 in code)
    '''
    transf_vecs = make_matrix_transformed(normalized_m, norms[:, np.newaxis], r_cut[i], p[i])

    return np.sum(transf_vecs, axis=0)

# @njit(parallel=True)
def create_V(normalized_m, norms, K=CFG.K):
    '''
    creates V
    '''
    V = []
    for i in range(K):
        V.append(
            create_V_i(i, normalized_m, norms)
        )

    return np.stack(V)

In [39]:
def force(r):
    '''
    r is a vector from one particle to another
    '''

    d = norm(r)

    f = 4 * (12 * pow(d, -13) - 6 * pow(d, -7)) * (r / d)

    # f = -10 * r

    return f

# @njit(
#     parallel=True,
#     fastmath=True
#     )
def _calculate_matrix_for_atom(relative_distances, rel_vel, r_cut=CFG.r_cut, p=CFG.p, N_neig=CFG.N_neig, K=CFG.K, use_orthogonal=False, use_A_t=True):
    '''

    relative_distances: np.array matrix of relative distance vectors

    '''
    
    # Only closest N_neig are counting:
    indexlist = np.argsort(norm(relative_distances, axis=1))

    relative_distances = relative_distances[indexlist[len(relative_distances) - N_neig:]]

    norms = norm(relative_distances, axis=-1)

    normalized_rel_distances = relative_distances / norms[:, np.newaxis]

    # ------СТАНДАРТНЫЙ РЕЖИМ:-------------------
    # V = create_V(normalized_rel_distances, norms)


    # if use_orthogonal and CFG.K == 3:
    #     diagonal_V_matr = [[0] * CFG.K for i in range(K)]
    #     for i in range(K):
    #         diagonal_V_matr[i][i] = V[i][i]
    #     V = np.array(
    #         diagonal_V_matr
    #     )
    # ----------------ДЛЯ 2 ЧАСТИЦ РЕЖИМ:----------------------------

    cross_vec = np.cross(relative_distances.squeeze(), rel_vel)
    cross_vec = cross_vec / norm(cross_vec)

    r = relative_distances.squeeze()

    V = np.vstack(
        [r,
        force(r),
        rel_vel
        # np.cross(relative_distances.squeeze(), rel_vel)
        ]
    )
    # print(relative_distances.squeeze(), rel_vel, np.cross(relative_distances, rel_vel))

    A = V / norm(V, axis=-1)[:, np.newaxis]

    # print(V)
    # print(get_pinv(A))

    if use_A_t:
        X = V @ A.T
    else:
        X = V

    return X, A

In [40]:
# @njit
def get_pinv(A):
    return np.linalg.pinv(A)

def get_matrix_for_atom(state, atom_number, N_neig=CFG.N_neig, use_orthogonal=True, use_A_t=True):
    '''

    This function will create X matrix for passed atom with
    arrays of r_cut and p of length k

    It is a wrapper for _get_relative_positions and _calculate_matrix_for_atom, so I can speed up matrix calculations
    with numba for _calculate_matrix_for_atom

    atom_number: a number of atom that we are passing
    row: one row from df_with_coords, i.e. df.iloc[index_of_row]

    '''

    # creating row of relative coordinates for concrete atom:
    relative_distances = _get_relative_positions(state=state, atom_number=atom_number)

    vel0 = state[0][1]
    vel1 = state[1][1]

    if atom_number == 0:
        rel_vel = vel0 - vel1
    else:
        rel_vel = vel1 - vel0

    X, A = _calculate_matrix_for_atom(relative_distances=relative_distances, rel_vel=rel_vel, N_neig=N_neig, use_orthogonal=use_orthogonal, use_A_t=use_A_t)

    flat_X = np.concatenate([X.flatten(), state[atom_number][1]])

    # print(
    #     state
    # )
    
    return flat_X, get_pinv(A)

# %timeit get_matrix_for_atom(row=df.iloc[0], atom_number=1)

In [41]:
csv_naming_coords = []
for i in range(CFG.N):
    csv_naming_coords.extend([str(i) + "x", str(i) + "y", str(i) + "z"])
result_coords_csv = "./integration_res/result_coords_n_from_2.csv"

csv_naming_vels = []
for i in range(CFG.N):
    csv_naming_vels.extend([str(i) + "vx", str(i) + "vy", str(i) + "vz"])
result_vels_csv = "./integration_res/result_vels_n_from_2.csv"


def fill_csv(state, path, mode="coords"):
    '''
    fills csv after current step
    '''
    s = ''
    if mode == "coords":
        idx = 0
    else:
        idx = 1
    with open(path, "a") as f:
        for atom_numb in range(CFG.N):
            s += ',' + ",".join(list(state[atom_numb][idx].astype(str)))
        f.write(s[1:] + '\n')

In [42]:
def make_step(n_state):
    pass

In [43]:
n_state[0]

[array([2.6166295 , 0.42700003, 3.75230512]),
 array([2.14348987, 0.4460663 , 0.8382205 ])]

**Короче: перемещения надо брать из unfolded координат, матрицы признаков надо брать из folded координат, а мб я и не прав**

In [44]:
# ЗАПУСКАЮ ЦИКЛ, СЧИТАЮ ВСЕ РАССТОЯНИЯ, ЕСЛИ ОНО МИНИМАЛЬНОЕ - СТРОЮ state для текущий частицы с этой частицей - делаю предсказание для этой частицы, но пока не сдвигаю, так делаю для всех частиц и затем сдвигаю

def make_predictions(n_state, model):
    Xs = []
    pinv_As = []
    for atom_num in range(N):
        state_2p = {}
        other_atom_for_2p_state = get_closest_atom(n_state, atom_num)
        # print(other_atom_for_2p_state)
        state_2p = {0: n_state[atom_num], 1: n_state[other_atom_for_2p_state]}
        # print(state_2p)
        #
        X, pinv_A = get_matrix_for_atom(state=state_2p, atom_number=0)  # в текущем стейте все время 0 атом - для которого предсказывать будем
        Xs.append(X)
        pinv_As.append(pinv_A)


    # print(Xs)

    Xs = np.array(Xs, dtype=np.float) if model_type != "network" else torch.tensor(np.array(Xs), dtype=torch.float)

    pinv_As = np.array(pinv_As)
    preds = descaler(model(Xs))

    return [pinv_As[i] @ preds[i] for i in range(CFG.N)]

In [45]:
make_predictions(n_state, model)

[array([1.86239076e-06, 1.50071894e-06, 1.15705657e-06]),
 array([-1.17380898e-10, -3.29429494e-10, -2.84651245e-10]),
 array([0., 0., 0.]),
 array([ 1.97696040e-106, -7.58777453e-107, -1.74887298e-107]),
 array([-8.78732575e-06, -1.04163483e-06, -2.06096658e-05]),
 array([ 6.49669253e-10, -2.59822511e-08,  7.80881698e-09]),
 array([-1.29939860e-106,  4.93421403e-107,  1.16956378e-107]),
 array([-1.33739126e-06, -4.16826726e-06,  5.01518250e-06]),
 array([-2.73088983e-08,  9.75004738e-08,  1.29814180e-07]),
 array([1.96315032e-07, 1.83714375e-07, 3.02853670e-07]),
 array([8.36725897e-07, 3.30492620e-06, 2.47271755e-06]),
 array([-4.11103109e-07, -1.92097237e-06, -1.45018881e-06]),
 array([ 5.72599184e-06,  5.88849505e-07, -6.96919976e-06]),
 array([-7.80543763e-05, -3.00748432e-05,  8.55637713e-05]),
 array([-4.64287547e-12,  4.37750356e-12,  8.59524889e-11]),
 array([ 2.76370072e-06, -2.08151530e-06, -1.56907245e-06]),
 array([-1.18380376e-17,  4.42647166e-20,  2.35743974e-17]),
 arra

In [46]:
def make_step(state, minus_vdt=minus_vdt):
    dses = make_predictions(state, model)
    dvs = make_predictions(state, model_vel)
    for atom_num in range(CFG.N):
        # print(state[atom_num][1] * dt)
        if minus_vdt:
            state[atom_num][0] += state[atom_num][1] * dt     # ЕСЛИ ВЫЧИТАЛИ vdt при обучении
        state[atom_num][0] += dses[atom_num]

        state[atom_num][1] += dvs[atom_num]
    
    return state

In [47]:
def integration_cycle(state, number_of_steps, xyz_path=None, minus_vdt=minus_vdt):
    '''
    state: starting state
    '''
    # print(state)
    for step in tqdm(range(number_of_steps)):
        fill_csv(state, result_coords_csv)
        fill_csv(state, result_vels_csv, mode="vels")

        state = make_step(state, minus_vdt=minus_vdt)

        fill_csv(state, result_coords_csv)

    return state

In [48]:
with open(result_coords_csv, "w") as f:
    f.truncate(0)
    csv.DictWriter(f, fieldnames=csv_naming_coords).writeheader()

with open(result_vels_csv, "w") as f:
    f.truncate(0)
    csv.DictWriter(f, fieldnames=csv_naming_coords).writeheader()

In [49]:
integration_cycle(state=n_state, number_of_steps=1000, minus_vdt=minus_vdt)
None

100%|██████████| 1000/1000 [00:20<00:00, 48.66it/s]


---
Проверка коэффициента диффузии: